# ![HS-Albsig Logo](https://www.hs-albsig.de/typo3conf/ext/hsas/Resources/Public/Images/logo.png)

<br><br><br>
<a class="anchor" id="Anfang"></a>
# Python: Applied Modells - Projektaufgabe

***
**Autoren:** Christian Seidler, Nils Heimbach


**Verlauf:**
Das Notebook wurde im Rahmen der Veranstaltung "Python: Applied Modells" des Studiengangs "Data Engineering and Consulting" der Hochschule Albstadt-Sigmaringen im Sommersemesters 2022 erstellt.
***

***
**Beschreibung:**


**Inhaltsverzeichnis:**
<ul>
<li><a href='#0'>0 Pre-Settings</a></li>
<li><a href='#1'>1 Import Bibliotheken</a></li>
<li><a href='#2'>2 Initialisieren des Run Directories und des Loggers</a></li>
<li><a href='#3'>3 Daten Vorbereitung</a></li>
    <ul>
        <li><a href='#3.1'>3.1 Daten einlesen</a></li>
        <li><a href='#3.2'>3.2 Daten zusammenfügen</a></li>
    </ul>
<li><a href='#4'>4 Visualisierung von CO2-Daten und Temperaturabweichung</a></li>
<li><a href='#5'>5 Ausreißeranalyse (Flats und Spikes)</a></li>
<li><a href='#6'>6 Regression</a></li>
<li><a href='#7'>7 Ende</a></li>
</ul>

<a class="anchor" id="0"></a>
# 0 Pre-Settings

In [1]:
# Importieren der Warnings Bibliothek
import warnings

# Warnungen ignorieren
warnings.filterwarnings("ignore")

In [2]:
# Erweiterung des Directories, um eigene Skripte importieren zu können.
import sys
code_directory = r"C:\Users\heimb\Documents\GitHub\AppliedModels\Code"

if code_directory not in sys.path:
    sys.path.append(code_directory)

In [3]:
%matplotlib qt

<ul>
<li><a href='#0'>Zurück zum Anfang</a></li>
<li><a href='#Anfang'>Zurück zum Inhaltsverzeichnis</a></li>
</ul>

<a class="anchor" id="1"></a>
# 1 Import Bibliotheken

In [4]:
# Importieren der allgemeinen Bibliotheken
import os
import logging
import pandas as pd

In [5]:
# Import der eigenen Skripte
import config as c
import load_data as load
import prepare_data as prep
import data_visualization as vis
import outlier_analysis as outlier
import regression as reg
import linear_regression_diagnostics as diagnostics
import logging_config as clog

<ul>
<li><a href='#1'>Zurück zum Anfang</a></li>
<li><a href='#Anfang'>Zurück zum Inhaltsverzeichnis</a></li>
</ul>

<a class="anchor" id="2'"></a>
# 2 Initialisieren des Run Directories und des Loggers

In [6]:
# Konfigurationsdatei laden
nils_filepath = r"C:\Users\heimb\Documents\GitHub\AppliedModels\\"
christian_filepath = r'D:\HS Albstadt\Sommersemester 2022' \
    r'\Python Applied Models\Abschlussaufgabe\\'

filepath = nils_filepath+r"Data\Input\Config.json"
config = c.load_config(filepath)

# Neues Run-Directory anlegen
run_directory = c.create_run_directory(config)

# Logger anlegen
clog.start_logger(run_directory=run_directory, config=config)

logging.info('Das Dashboard wurde gestartet.')
logging.info('Die Konfigurationsparameter wurden erfolgreich eingelesen.')
logging.info('Das Run-Directory wurde erfolgreich angelegt.')

<ul>
<li><a href='#2'>Zurück zum Anfang</a></li>
<li><a href='#Anfang'>Zurück zum Inhaltsverzeichnis</a></li>
</ul>

<a class="anchor" id="3"></a>
# 3 Daten Vorbereitung

<a class="anchor" id="3.1"></a>
## 3.1 Daten einlesen

In [7]:
# Fossil-Fuel-Daten einlesen
df_fossil_fuel = load.load_fossil_fuel_data_to_dataframe(config)
df_fossil_fuel = prep.prepare_fossil_fuel_dataframe(config, df_fossil_fuel)
print(df_fossil_fuel.shape)
logging.info('Die CO2-Daten wurden erfolgreich eingelesen.')

# Temperaturdaten einlesen
df_temperature = load.load_temperature_dataframe(config)
df_temperature = prep.prepare_temperature_dataframe(config, df_temperature)
print(df_temperature.shape)
logging.info('Die mittlere Temperaturabweichungsdaten wurden '
             'erfolgreich eingelesen.')

(223, 3)
(137, 2)


<a class="anchor" id="3.2"></a>
## 3.2 Daten zusammenfügen

In [8]:
# Datensätze zusammenfügen
df_merged = pd.merge(df_fossil_fuel, df_temperature, how='inner',
                     on='Year')
print(df_merged)
logging.info('Die Datensätze wurden erfolgreich zusammengeführt.')

# Tabelle zur Validierung in Datei abspeichern (.csv)
filepath = os.path.join(run_directory, 'merged_dataset.csv')
df_merged.to_csv(filepath)
logging.info('Der zusammengeführte Datensatz wurde unter %s'
             ' gespeichert.', filepath)

     Year  Country   Total      Mean
0    1880  GERMANY   33980 -0.204167
1    1881  GERMANY   36644 -0.115833
2    1882  GERMANY   39210 -0.102500
3    1883  GERMANY   41783 -0.210000
4    1884  GERMANY   42993 -0.284167
..    ...      ...     ...       ...
130  2010  GERMANY  206943  0.713333
131  2011  GERMANY  199754  0.602500
132  2012  GERMANY  201762  0.635000
133  2013  GERMANY  206521  0.655833
134  2014  GERMANY  196314  0.743333

[135 rows x 4 columns]


<ul>
<li><a href='#3'>Zurück zum Anfang</a></li>
<li><a href='#Anfang'>Zurück zum Inhaltsverzeichnis</a></li>
</ul>

<a class="anchor" id="4"></a>
# 4 Visualisierung von CO2-Daten und Temperaturabweichung

In [9]:
# Verlauf von Temperatur
vis.show_scatterplot(x='Year', y='Mean', data=df_merged,
                     title='Year vs. Mean')
vis.savefig(run_directory, 'Scatterplot Year vs Mean')
logging.info('Scatterplot der Temperatur wurde erstellt und'
             ' abgespeichert.')

# Temperaturwerte über 800 entfernen (Jahre 1928 & 1984)
df_final = df_merged[(df_merged.Mean) < 800]
logging.info('Ausreißer des Means über 800 werden entfernt.')

# Verlauf der Temperatur
vis.show_scatterplot(x='Year', y='Mean', data=df_final,
                     title='Year vs. Mean after outlier removal')
vis.savefig(run_directory, 'Scatterplot Year vs Mean after outlier '
            'removal')
logging.info('Scatterplot der Temperatur ohne Ausreißer wurde erstellt und'
             ' abgespeichert.')

# Verlauf von CO2-Emissionen
vis.show_scatterplot(x='Year', y='Total', data=df_final,
                     title='Year vs. Total CO2-Emissions of Germany')
vis.savefig(run_directory, 'Scatterplot Year vs Total CO2-Emissions of '
            'Germany')
logging.info('Scatterplot der CO2-Emissionen wurde erstellt und '
             'abgespeichert.')

# Scatterplot Temperatur vs. CO2-Emissionen
vis.show_scatterplot(x='Total', y='Mean', data=df_final,
                     title='Yearly Temperature Deviation vs.'
                     'Total CO2-Emissions of Germany')
vis.savefig(run_directory, 'Scatterplot Yearly Temperature Deviation vs'
            'Total CO2-Emissions of Germany')
logging.info('Scatterplot der CO2-Emissionen und Temperatur wurde erstellt'
             ' und abgespeichert.')

# Regressionsplot
vis.show_regressionplot(x='Total', y='Mean', data=df_final,
                        title='Yearly Temperature Deviation vs.'
                        'Total CO2-Emissions of Germany')
vis.savefig(run_directory, 'Yearly Temperature Deviation vs Total '
            'CO2-Emissions of Germany ')
logging.info('Scatterplot der jährlichen Temepratur Abweichung vs. CO2 '
             '-Emissionen wurde erstellt und abgespeichert.')

<ul>
<li><a href='#4'>Zurück zum Anfang</a></li>
<li><a href='#Anfang'>Zurück zum Inhaltsverzeichnis</a></li>
</ul>

<a class="anchor" id="5"></a>
# 5 Ausreißeranalyse (Flats und Spikes)

In [10]:
# Boxplot
outlier.show_boxplot(y='Mean', data=df_final,
                     title='Boxplot for average yearly'
                     ' temperature deviation')
vis.savefig(run_directory, 'Boxplot for average yearly temperature '
            'deviation')
logging.info('Boxplot der durchschnittlichen jährlichen Temeperatur '
             'Abweichung wurde erstellt und abgespeichert.')

outlier.show_boxplot(y='Total', data=df_final,
                     title='Boxplot for total CO2-Emissions of Germany'
                     ' per year')
vis.savefig(run_directory, 'Boxplot for total CO2-Emissions of Germany'
            ' per year')
logging.info('Boxplot der gesamten CO2-Emissionen von Deutschland '
             'wurde erstellt und abgespeichert.')

# Histogramme
outlier.show_histogramm(x='Mean', data=df_final,
                        title='Average yearly temperature deviation')
vis.savefig(run_directory, 'Average yearly temperature deviation')
logging.info('Histogram der durchschnittlichen jährlichen Temperatur '
             ' Abweichungen wurde erstellt und abgespeichert.')

outlier.show_histogramm(x='Total', data=df_final,
                        title='Average total CO2-Emissions of Germany'
                        ' per year')
vis.savefig(run_directory, 'Average total CO2-Emissions of Germany'
            ' per year')
logging.info('Histogram der gesamten CO2-Emissionen von Deutschland '
             'Abweichungen wurde erstellt und abgespeichert.')

# Ausreißer (Spikes) mittels Z-Score
logging.info('Ausreißer (Spikes) werden mit Hilfe des Z-Score für die '
             'Spalte "Mean" errechnet.')
outlier.calc_and_print_outliers_with_z_score(data=df_final, column='Mean')

logging.info('Ausreißer (Spikes) werden mit Hilfe des Z-Score für die '
             'Spalte "Total" errechnet.')
outlier.calc_and_print_outliers_with_z_score(data=df_final, column='Total')

# Ausreißer (Spikes) mittels Interquartilabstand
logging.info('Ausreißer (Spikes) werden mit Hilfe des Interquartilabstands'
             '  für die Spalte "Total" errechnet.')
outlier.calc_and_print_outliers_with_iqr(data=df_final, column='Mean')
logging.info('Ausreißer (Spikes) werden mit Hilfe des Interquantilabstands'
             ' für die Spalte "Total" errechnet.')
outlier.calc_and_print_outliers_with_iqr(data=df_final, column='Total')

# Suche nach Flats
logging.info('Die Anzahl an Flats der Spalte "Mean" werden berechnet und '
             'geprintet.')
outlier.print_num_of_flats(data=df_final, column='Mean')

logging.info('Die Anzahl an Flats der Spalte "Total" werden berechnet und '
             'geprintet.')
outlier.print_num_of_flats(data=df_final, column='Total')

# Abspeichern zu Validierungszwecken
filepath = os.path.join(run_directory, 'final_dataset.csv')
df_final.to_csv(filepath)
logging.info('Der finale Dataframe wurde abgespeichert.')


Outliers in Column Mean: (array([], dtype=int64),)
Outliers in Column Total: (array([], dtype=int64),)
Upper Outliers in Column Mean: (array([], dtype=int64),)
Lower Outliers in Column Mean: (array([], dtype=int64),)
Upper Outliers in Column Total: (array([], dtype=int64),)
Lower Outliers in Column Total: (array([], dtype=int64),)
Flats in column Mean: Empty DataFrame
Columns: [Year, Country, Total, Mean, Mean_diff]
Index: []
Flats in column Total: Empty DataFrame
Columns: [Year, Country, Total, Mean, Total_diff]
Index: []


<ul>
<li><a href='#5'>Zurück zum Anfang</a></li>
<li><a href='#Anfang'>Zurück zum Inhaltsverzeichnis</a></li>
</ul>

<a class="anchor" id="6"></a>
# 6 Regression

In [11]:
# Durchführen einer Regression (-1, um Intercept zu entfernen
# I(Total**3) für kubischen Term)
models = config["Regression_Models"]
for key, value in models.items():
    logging.info('Berechnung für Model %s gestartet.', key)
    logging.info('Parameter: Formel: %s, Teilzeitraum: %s, '
                 'Validation Sample: %s', value['Formula'],
                 value['Teil_Zeitraeume'], value['Validation_Sample'])
    # Unterverzeichnis anlegen
    model_directory = c.create_model_directory(run_directory, key)
    logging.info('Das Model-Directory wurde erfolgreich angelegt.')

    df_teil_zeitraum = df_final
    df_validation = None

    # Teilzeitraum bestimmen
    teil_zeitraum = reg.get_teil_zeitraum_from_config(
        value['Teil_Zeitraeume'])
    # Datensatz nach Teilzeitraum filtern
    if teil_zeitraum is not None:
        df_teil_zeitraum = df_final[df_final['Year'].isin(teil_zeitraum)]
        logging.info('Teilzeitraum erfolgreich angelegt.')

    # Validation Sample
    teil_zeitraum = reg.get_teil_zeitraum_from_config(
        value['Validation_Sample'])
    # Extra Datensatz für Validierungssample
    if teil_zeitraum is not None:
        df_validation = df_final[df_final['Year'].isin(teil_zeitraum)]
        x_val = df_validation['Total']
        logging.info('Validation Sample erstellt.')

    # Regression durchführen
    regression_model = reg.fit_regression_model(config=value,
                                                data=df_teil_zeitraum)
    logging.info('Regressionsmodell erfolgreich erstellt.')

    # Validierung durchführen
    if df_validation is not None:
        predictions = regression_model.predict(x_val)
        # Grafische Darstellung
        vis.visualize_validation_sample(df_validation, predictions)
        vis.savefig(model_directory, 'Validation_Results')
        logging.info('Validation durchgeführt.')

    logging.info('Darstellung der Regressionsergebnisse.')
    print(regression_model.summary())
    logging.info(regression_model.summary())

    # Grafische Darstellung des Ergebnisses
    vis.visualize_regression_results(df_teil_zeitraum, regression_model)
    vis.savefig(model_directory, 'Regression_Results')

    # Abspeichern der Regressionsergebnisse als .csv
    filename = str(key) + '_Regression'
    reg.save_regression_results_as_csv(model=regression_model,
                                       run_directory=model_directory,
                                       filename=filename+'_Results.csv')

    # Gesamte Summary abspeichern
    reg.save_regression_summary_as_csv(model=regression_model,
                                       run_directory=model_directory,
                                       filename=filename+'_Summary.csv')

    # Erweiterung um Überprüfung der Annahmen der Regression
    cls = diagnostics.Linear_Reg_Diagnostic(regression_model)
    cls()
    vis.savefig(model_directory, 'model_assumptions')
    logging.info('Plots zur Überprüfung der Modellannahmen erfolgreich'
                 ' erstellt und abgespeichert.')

                            OLS Regression Results                            
Dep. Variable:                   Mean   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.358
Method:                 Least Squares   F-statistic:                     74.51
Date:                Thu, 07 Jul 2022   Prob (F-statistic):           1.77e-14
Time:                        22:41:50   Log-Likelihood:                -3.3847
No. Observations:                 133   AIC:                             10.77
Df Residuals:                     131   BIC:                             16.55
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3867      0.051     -7.585      0.0

                            OLS Regression Results                            
Dep. Variable:                   Mean   R-squared:                       0.300
Model:                            OLS   Adj. R-squared:                  0.246
Method:                 Least Squares   F-statistic:                     5.563
Date:                Thu, 07 Jul 2022   Prob (F-statistic):             0.0347
Time:                        22:42:13   Log-Likelihood:                 19.744
No. Observations:                  15   AIC:                            -35.49
Df Residuals:                      13   BIC:                            -34.07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4064      0.334      4.205      0.0

<ul>
<li><a href='#6'>Zurück zum Anfang</a></li>
<li><a href='#Anfang'>Zurück zum Inhaltsverzeichnis</a></li>
</ul>

<a class="anchor" id="7"></a>
# 7 Ende

In [12]:
logging.info('Ende des Programms.')
# Logger abschalten
logging.shutdown()

<ul>
<li><a href='#7'>Zurück zum Anfang</a></li>
<li><a href='#Anfang'>Zurück zum Inhaltsverzeichnis</a></li>
</ul>